In [0]:
from google.colab import drive

drive.mount('/gdrive')
gdrive_root = '/gdrive/My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
 
batch_size = 100
 
train_dataset = datasets.MNIST(root='./mnist_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
 
test_dataset = datasets.MNIST(root='./mnist_data/',
                             train=False,
                             transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels = 64, kernel_size=8, stride = 2, padding = 3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels = 128, kernel_size=6, stride = 2)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels = 128, kernel_size=5, stride = 1)
        self.dropout2 = nn.Dropout(0.5)
        self.FC1 = nn.Linear(128, 10)
        
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.dropout1(x)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.dropout2(x)
        x = x.view(in_size, -1)
        x = self.FC1(x)

        return x

In [0]:
model = Net()
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

best_acc = 0.
ckpt_dir = os.path.join(gdrive_root, 'checkpoints')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_path = os.path.join(ckpt_dir, 'modelB_ckpt.pt')
if os.path.exists(ckpt_path):
    ckpt = torch.load(ckpt_path)
    try:
        model.load_state_dict(ckpt['model'])
        optimizer.load_state_dict(ckpt['optimizer'])
        best_acc = ckpt['best_acc']
    except RuntimeError as e:
        print('wrong checkpoint')
    else:    
        print('checkpoint is loaded !')
        print('current best accuracy : %.2f' % best_acc)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

def test(best_acc):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        
        test_loss += criterion(output, target).item()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    test_acc = correct / float(len(test_loader.dataset))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

    if test_acc > best_acc:
        best_acc = test_acc
        ckpt = {'model': model.state_dict(),
                'optimizer':optimizer.state_dict(),
                'best_acc':best_acc}
        torch.save(ckpt, ckpt_path)
        print('checkpoint is saved!')

In [0]:
for epoch in range(1, 10):
    train(epoch)
    test(best_acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301122
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.294665
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.290123
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.287006
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.269048
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.269648
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.250512
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.239052
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.203217
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.199121
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.132904
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.103691
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.024667
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.944750
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.838099
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.726280
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.527734
Train Epoch: 1 [17000/60000 (28%)]	Loss: 1.431014
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.379140
Train Epoch: 1 [19000/60000 (32%)]	Loss: 1.094452
Train Epoch: 1 [200

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0032, Accuracy: 9131/10000 (91%)

checkpoint is saved!
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.461952
Train Epoch: 2 [1000/60000 (2%)]	Loss: 0.469936
Train Epoch: 2 [2000/60000 (3%)]	Loss: 0.457527
Train Epoch: 2 [3000/60000 (5%)]	Loss: 0.461588
Train Epoch: 2 [4000/60000 (7%)]	Loss: 0.534066
Train Epoch: 2 [5000/60000 (8%)]	Loss: 0.402420
Train Epoch: 2 [6000/60000 (10%)]	Loss: 0.373896
Train Epoch: 2 [7000/60000 (12%)]	Loss: 0.462134
Train Epoch: 2 [8000/60000 (13%)]	Loss: 0.335636
Train Epoch: 2 [9000/60000 (15%)]	Loss: 0.445530
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.283863
Train Epoch: 2 [11000/60000 (18%)]	Loss: 0.648037
Train Epoch: 2 [12000/60000 (20%)]	Loss: 0.488870
Train Epoch: 2 [13000/60000 (22%)]	Loss: 0.454585
Train Epoch: 2 [14000/60000 (23%)]	Loss: 0.359243
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.377052
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.578992
Train Epoch: 2 [17000/60000 (28%)]	Loss: 0.412574
Train Epoch: 2 [18000/60000 (30%)]	Lo